In [1]:
import ee
import geopandas as gpd
import numpy as np
import pandas as pd

import agrigee_lite as agl

np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)
pd.set_option('display.float_format', '{:.2f}'.format)

# pandas show all columns
pd.set_option('display.max_columns', None)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project="ee-paulagibrim")

In [4]:
gdf = gpd.read_parquet("MG_crops.parquet")

gdf.sample(5, random_state=42)

,geometry,varda_id,ha,CD_MUN,CD_MICRO,CD_MESO,percentage,year,crop_class,start_date,end_date
804092,"POLYGON ((-47.57329 -19.95601, -47.57352 -19.9...",019114bc-78c3-7c66-bf0f-f0c16501be5f,5.20,3118205,31022,3105,0.97,2020,Sugar Cane,2019-10-01,2020-10-01
1074092,"POLYGON ((-45.82927 -21.72358, -45.82867 -21.7...",01911462-e721-7cac-9f97-f03e1d48f002,1.20,3139003,31049,3110,1.00,2022,Other Temporary Crops,2021-10-01,2022-10-01
935752,"POLYGON ((-46.23982 -21.37212, -46.23895 -21.3...",01911444-f902-7577-b7e9-a5672938b6cf,2.38,3143005,31048,3110,0.93,2020,Other Temporary Crops,2019-10-01,2020-10-01
831325,"POLYGON ((-46.31852 -16.87922, -46.31671 -16.8...",01911dcf-f4d5-7d06-9c1c-f1231c4b1b39,4.42,3170404,31001,3101,0.99,2023,Sugar Cane,2022-10-01,2023-10-01
338142,"POLYGON ((-46.30273 -21.2375, -46.30173 -21.23...",0191142a-10c8-79fb-85a8-8cffe3966b6d,1.40,3143005,31048,3110,0.97,2020,Coffee,2019-10-01,2020-10-01


In [5]:
gdf["start_date"] = pd.to_datetime(gdf["start_date"])
gdf["end_date"] = pd.to_datetime(gdf["end_date"])

In [8]:
sampled_gdf = gdf.sample(250, random_state=42).reset_index(drop=True)

results = agl.get.multiple_sits(sampled_gdf, agl.sat.Landsat8(bands=["red"]), date_types=["timestamp"], force_redownload=True)

Simplifying clusters: 100%|██████████| 1/1 [00:00<00:00,  6.57it/s]

['indexnum', 'l8sr_observations', 'l8sr_timestamp_0', 'l8sr_timestamp_1', 'l8sr_timestamp_2', 'l8sr_timestamp_3', 'l8sr_timestamp_4', 'l8sr_timestamp_5', 'l8sr_timestamp_6', 'l8sr_timestamp_7', 'l8sr_timestamp_8', 'l8sr_timestamp_9', 'l8sr_timestamp_10', 'l8sr_timestamp_11', 'l8sr_timestamp_12', 'l8sr_timestamp_13', 'l8sr_timestamp_14', 'l8sr_timestamp_15', 'l8sr_timestamp_16', 'l8sr_timestamp_17', 'l8sr_red_0', 'l8sr_red_1', 'l8sr_red_2', 'l8sr_red_3', 'l8sr_red_4', 'l8sr_red_5', 'l8sr_red_6', 'l8sr_red_7', 'l8sr_red_8', 'l8sr_red_9', 'l8sr_red_10', 'l8sr_red_11', 'l8sr_red_12', 'l8sr_red_13', 'l8sr_red_14', 'l8sr_red_15', 'l8sr_red_16', 'l8sr_red_17', 'l8sr_timestamp_18', 'l8sr_timestamp_19', 'l8sr_timestamp_20', 'l8sr_timestamp_21', 'l8sr_timestamp_22', 'l8sr_timestamp_23', 'l8sr_timestamp_24', 'l8sr_timestamp_25', 'l8sr_timestamp_26', 'l8sr_timestamp_27', 'l8sr_timestamp_28', 'l8sr_timestamp_29', 'l8sr_timestamp_30', 'l8sr_timestamp_31', 'l8sr_timestamp_32', 'l8sr_timestamp_33', 'l

In [9]:
sampled_gdf_w_results = sampled_gdf.join(results)

In [12]:
sampled_gdf_w_results[sampled_gdf_w_results.l8sr_observations==0].ha.max()

np.float64(3.7641220605804127)